In [1]:
%load_ext autoreload
%autoreload 2
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim
from EEG_Encoder.net_generator import Classifier
from torchinfo import summary

In [2]:
# Default Options
length = 512
channel = 96
min_CNN = 200
n_classes = 40
classes = range(n_classes)
classifier= ""
GPUindex=0
kind="from-scratch"

In [3]:
# Options for EEGNet
classifier="EEGNet"
net, nonclasses = Classifier(
                    n_classes,
                    classes,
                    classifier,
                    GPUindex,
                    length,
                    channel,
                    min_CNN,
                    kind)
summary(net, input_size=(1, 96, 512))

DONE: CREATE TORCH CLASSIFIER
classifier_EEGNet(
  (network): Sequential(
    (0): ZeroPad2d((128, 127, 0, 0))
    (1): Conv2d(1, 8, kernel_size=(1, 256), stride=(1, 1))
    (2): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(8, 16, kernel_size=(96, 1), stride=(1, 1), groups=8)
    (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ELU(alpha=1.0)
    (6): AvgPool2d(kernel_size=(1, 4), stride=(1, 4), padding=0)
    (7): Dropout(p=0.5, inplace=False)
    (8): ZeroPad2d((8, 7, 0, 0))
    (9): Conv2d(16, 16, kernel_size=(1, 16), stride=(1, 1), groups=8)
    (10): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1))
    (11): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ELU(alpha=1.0)
    (13): AvgPool2d(kernel_size=(1, 8), stride=(1, 8), padding=0)
    (14): Dropout(p=0.5, inplace=False)
  )
  (fc): Linear(in_features=256, out_features=40, bias=True)
)
Input 

Layer (type:depth-idx)                   Output Shape              Param #
classifier_EEGNet                        [1, 40]                   --
├─Sequential: 1-1                        [1, 16, 1, 16]            --
│    └─ZeroPad2d: 2-1                    [1, 1, 96, 767]           --
│    └─Conv2d: 2-2                       [1, 8, 96, 512]           2,056
│    └─BatchNorm2d: 2-3                  [1, 8, 96, 512]           16
│    └─Conv2d: 2-4                       [1, 16, 1, 512]           1,552
│    └─BatchNorm2d: 2-5                  [1, 16, 1, 512]           32
│    └─ELU: 2-6                          [1, 16, 1, 512]           --
│    └─AvgPool2d: 2-7                    [1, 16, 1, 128]           --
│    └─Dropout: 2-8                      [1, 16, 1, 128]           --
│    └─ZeroPad2d: 2-9                    [1, 16, 1, 143]           --
│    └─Conv2d: 2-10                      [1, 16, 1, 128]           528
│    └─Conv2d: 2-11                      [1, 16, 1, 128]           272
│    └─

In [12]:
# Options for Stacked_BiLSTM
classifier="Stacked_BiLSTM"
net, nonclasses = Classifier(
                    n_classes,
                    classes,
                    classifier,
                    GPUindex,
                    length,
                    channel,
                    min_CNN,
                    kind)
summary(net, input_size=(1, 96, 512))

DONE: CREATE TORCH CLASSIFIER
classifier_Stacked_BiLSTM(
  (stacked_bilstm): LSTM(96, 128, num_layers=2, batch_first=True, bidirectional=True)
  (output1): Linear(in_features=256, out_features=128, bias=True)
  (relu): ReLU()
  (output2): Linear(in_features=128, out_features=40, bias=True)
)


Layer (type:depth-idx)                   Output Shape              Param #
classifier_Stacked_BiLSTM                [1, 40]                   --
├─LSTM: 1-1                              [1, 512, 256]             626,688
├─Linear: 1-2                            [1, 128]                  32,896
├─ReLU: 1-3                              [1, 128]                  --
├─Linear: 1-4                            [1, 40]                   5,160
Total params: 664,744
Trainable params: 664,744
Non-trainable params: 0
Total mult-adds (M): 320.90
Input size (MB): 0.20
Forward/backward pass size (MB): 1.05
Params size (MB): 2.66
Estimated Total Size (MB): 3.91

In [3]:
# Options for CNN
classifier="CNN"
net, nonclasses = Classifier(
                    n_classes,
                    classes,
                    classifier,
                    GPUindex,
                    length,
                    channel,
                    min_CNN,
                    kind)
summary(net, input_size=(1, 96, 512))

DONE: CREATE TORCH CLASSIFIER
classifier_CNN(
  (conv1): Conv1d(1, 8, kernel_size=(32,), stride=(1,))
  (fc1): Linear(in_features=768, out_features=40, bias=True)
  (pool1): AvgPool1d(kernel_size=(128,), stride=(64,), padding=(0,))
  (activation): ELU(alpha=1.0)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=240, out_features=40, bias=True)
)
Input size:  torch.Size([1, 96, 512])
After unsquueze:  torch.Size([1, 96, 1, 512])
After view:  torch.Size([96, 1, 512])
After conv1:  torch.Size([96, 8, 481])
After view 2:  torch.Size([1, 96, 8, 481])
After permute 2:  torch.Size([1, 481, 96, 8])
After view 3:  torch.Size([1, 481, 768])
After fc1:  torch.Size([1, 481, 40])
After permute 3:  torch.Size([1, 40, 481])
After pool1:  torch.Size([1, 40, 6])
After view 4:  torch.Size([1, 240])
After fc2:  torch.Size([1, 40])


Layer (type:depth-idx)                   Output Shape              Param #
classifier_CNN                           [1, 40]                   --
├─Conv1d: 1-1                            [96, 8, 481]              264
├─ELU: 1-2                               [96, 8, 481]              --
├─Dropout: 1-3                           [1, 481, 768]             --
├─Linear: 1-4                            [1, 481, 40]              30,760
├─Dropout: 1-5                           [1, 481, 40]              --
├─AvgPool1d: 1-6                         [1, 40, 6]                --
├─Linear: 1-7                            [1, 40]                   9,640
Total params: 40,664
Trainable params: 40,664
Non-trainable params: 0
Total mult-adds (M): 12.23
Input size (MB): 0.20
Forward/backward pass size (MB): 3.11
Params size (MB): 0.16
Estimated Total Size (MB): 3.47